Решить задачу машинного перевода выбрав свой язык:
1. Формируем датасет с исходного языка на целевой (код прописать в классе)(английский-итальянский)
2. Строим архитектуру нейронной сети 
3. Обучаем 
4. Проверить качество с помощью метрики BLEU


In [2]:
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Подготовка данных

In [4]:
#Определим по умолчанию 2 токена которые будут нам информировать о начале предложения и конце предложения (SOS и EOS):
SOS_token = 0
EOS_token = 1


In [5]:
# Создадим объект словаря нашего языка, который будет хранить данные по маппингу слов - index2word и обратно word2index и плюс 
#второстепенные методы по добавлению токена и обработке предложений:
class LanguageVocabulary(object):
    def __init__(self, name):
        # название языка
        self.name = name
        # словарик word2index который хранит соответственно кодировку слова в целочисленный индекс словаря
        self.word2index = {}
        # обычный словарик который хранит распределение слов, сколько слов мы использовали и сколько обнаружили
        self.word2count = {}
        # Обратный словарик словарю word2index где хранятся уже индексы и замаппенные слова к каждому индексу,
        #нужен будет для расшифровки последовательности
        self.index2word = {0: "SOS", 1: "EOS"}
        # Count SOS and EOS, храним просто общее количество слов в нашем словаре, то есть количество токенов в сформированном словарике нашего языка
        self.n_words = 2

    def add_sentence(self, sentence):
        """
        Метод класса, для добавления предложения в словарь.
        Каждое предложение поступающее к нам, будет разбираться на
        примитивные токены и добавляться в словарь при помощи метода класса addword()
        """
        for word in sentence.split(' '):
            self.add_word(word)


    def add_word(self, word):
        # проверяем не входит ли наше слово в словарь word2index
        if word not in self.word2index:
            # добавляем в качестве ключа слово а в качестве значения последнее n_words
            self.word2index[word] = self.n_words
            # меняем на единичку
            self.word2count[word] = 1
            # и соответственно меняем и index2word словарик добавляя уже слово для декодирования
            self.index2word[self.n_words] = word
            # инкрементируем n_words
            self.n_words += 1
        else:
            # Если такое уже слово есть просто добавляем 1 что добавилось одно слово
            self.word2count[word] += 1

In [6]:
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalize_string(s):
    # Декодируем из юникода в ascii, переводим в нижний регистр, убираем пробелы
    s = unicode_to_ascii(s.lower().strip())
    # Что означает данное регулярное выражение - точку, !, ? меняем на пробел чтобы этот символ стоял отдельно от всех
    s = re.sub(r"([.!?])", r" \1", s)
    # оставляем только наборы символов указанных в паттерне регулярного выражения остальное заменим на пробел
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

read_languages - функция открывает документы и создает словарь

In [7]:
def read_languages(lang1, lang2, reverse=False):
    print("Reading lines...")
    # Берем документ корпуса, лежащий в директории  подставляя значения указанных языков  eng-it, он читается и разбивается на предложения
    lines = open('D:/файлы/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    # Разбиваем построчно и нормализуем строку:
    pairs = [[normalize_string(s) for s in l.split('\t')] for l in lines]
    # Можем создавать и проходить как с целевого языка на исходный так и наоборот:
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = LanguageVocabulary(lang2)
        output_lang = LanguageVocabulary(lang1)
    else:
        input_lang = LanguageVocabulary(lang1)
        output_lang = LanguageVocabulary(lang2)
    return input_lang, output_lang, pairs

Для упрощения отберем фразы, которые будут начинаться на биграммы указанные в eng_prefixes и  ограничим длину предложения документа 10 символами

In [9]:
MAX_LENGTH = 10
eng_prefixes = (
   # "i am ", "i m ",
   # "he is", "he s ",
    "she is", "she s",
  #  "you are", "you re ",
   "we are", "we re ",
   # "they are", "they re "
)

def filter_pair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]

prepare_data - функция возвращает данные

In [11]:
def prepare_data(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = read_languages(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filter_pairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [12]:
#5161 пар
input_lang, output_lang, pairs = prepare_data('eng', 'it', True)
print(random.choice(pairs))

Reading lines...
Read 377937 sentence pairs
Trimmed to 5161 sentence pairs
Counting words...
Counted words:
it 2110
eng 1511
['e sempre occupata .', 'she is always busy .']


Encoder

In [14]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        #input_size -размер словаря
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        #Приводим эмбеддинг к формату одного предлоежния 1х1 и любая размерность
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Decoder

In [16]:
class AttentionDecoder(nn.Module):
    #  на вход мы получаем размер скрытого представления
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttentionDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Вспомогательные функции для работы с полученной репрезентацией и для кодирования для подачи в модель

In [18]:
# Токены кодируем в целочисленное представление
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


# Берем предложение с указанным языком, делаем из него индексы и вставляем метку конца предложения, превращаем в тензор:
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# Для создания тензора из пар:
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [19]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    #  инициализируем скрытое представление для энкодера
    encoder_hidden = encoder.initHidden()
    # Скиыдваем градиенты для алгоритма градиентного спуска для энкодера  и  дэкодера
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Получаем размер в словаря (токенов) для входящего и выходящего тензора, пробегаемся по каждому предложению по кусочкам
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    # Создаем переменную где будем хранить наши выходы из энкодера 
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    # пробегаем по длине входящего тензора и в экодер передаем последовательно каждый из токенов:
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        # Сохраняем все выходы из энкодера для одного слова (для передачи в декодер, ниже)
        encoder_outputs[ei] = encoder_output[0, 0]


    # Закончили с энкодером пошли к декодеру, как было сказано декодер начинается с SOS
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    # Будем использовать Teacher Forcing в части случае (подставляя правильную последовательность)
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        # Подаем decoder_input = torch.tensor([[SOS_token]], device=device) то есть по одному слову и скрытое представление
        for di in range(target_length):
            # Переведенное предложение и скрытое представление
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Считаем ошибку
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

In [20]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / percent
    rs = es - s
    return '%s (- eta: %s)' % (asMinutes(s), asMinutes(rs))

Функция которая будет пробегаться по всем парам и использовать эти пары для обучения сети

In [22]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    # Делаем выборку наших пар функцией которую создали до
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    # FYI! Используем Negative Log-Likelihood Loss потому что log softmax уже присутствует в модели
    criterion = nn.NLLLoss()

    for epoch in range(1, n_iters + 1):
        training_pair = training_pairs[epoch - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        # функция для тренировки на отдельных токенах, написали выше
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_iters),
                                         epoch, epoch / n_iters * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    showPlot(plot_losses)

In [23]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.switch_backend('agg')


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Функция, которая позволит использовать Encoder-Decoder для перевода предложения

In [25]:
def evaluate_sentences(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [26]:
import evaluate

bleu = evaluate.load("bleu")

references, predictions = [],[]
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        references.append(pair[1])
        output_words, attentions = evaluate_sentences(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        predictions.append(output_sentence)
        print('')

    results = bleu.compute(predictions=predictions, references=references)
    return f"BLEU Score: {results}"

bleu = evaluate.load("bleu")

references, predictions = [],[]
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        references.append(pair[1])
        output_words, attentions = evaluate_sentences(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        predictions.append(output_sentence)
        print('')

    results = bleu.compute(predictions=predictions, references=references)
    return f"BLEU Score: {results}"

Обучение

In [28]:
hidden_size = 512
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = AttentionDecoder(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
trainIters(encoder1, decoder1, 50000, print_every=500)

0m 43s (- eta: 71m 29s) (500 1%) 3.1737
1m 21s (- eta: 66m 22s) (1000 2%) 2.8045
2m 0s (- eta: 64m 45s) (1500 3%) 2.7319
2m 38s (- eta: 63m 26s) (2000 4%) 2.6279
3m 17s (- eta: 62m 27s) (2500 5%) 2.5381
3m 55s (- eta: 61m 34s) (3000 6%) 2.4116
4m 35s (- eta: 60m 56s) (3500 7%) 2.3713
5m 13s (- eta: 60m 8s) (4000 8%) 2.2564
5m 52s (- eta: 59m 27s) (4500 9%) 2.1489
6m 31s (- eta: 58m 47s) (5000 10%) 2.0499
7m 10s (- eta: 58m 6s) (5500 11%) 1.9616
7m 48s (- eta: 57m 12s) (6000 12%) 1.9587
8m 26s (- eta: 56m 30s) (6500 13%) 1.8358
9m 4s (- eta: 55m 46s) (7000 14%) 1.7218
9m 59s (- eta: 56m 38s) (7500 15%) 1.7228
10m 38s (- eta: 55m 50s) (8000 16%) 1.4883
11m 16s (- eta: 55m 0s) (8500 17%) 1.4491
11m 54s (- eta: 54m 16s) (9000 18%) 1.3857
12m 38s (- eta: 53m 55s) (9500 19%) 1.3255
13m 24s (- eta: 53m 36s) (10000 20%) 1.2350
14m 10s (- eta: 53m 19s) (10500 21%) 1.2068
14m 49s (- eta: 52m 32s) (11000 22%) 1.1035
15m 28s (- eta: 51m 47s) (11500 23%) 1.0280
16m 17s (- eta: 51m 34s) (12000 24%) 

In [29]:
score = evaluateRandomly(encoder1, decoder1, n=1000)

> abbiamo finito le munizioni .
= we re out of ammunition .
< we re out of ammo . <EOS>

> canto piuttosto bene .
= she sang pretty well .
< she sang pretty well . <EOS>

> siamo da incolpare .
= we re to blame .
< we re to blame . <EOS>

> noi siamo sempre impegnate .
= we re always busy .
< we re always busy . <EOS>

> troveremo una cura .
= we re going to find a cure .
< we re going to find a cure . <EOS>

> siamo toste .
= we re tough .
< we re tough . <EOS>

> lei e una strega .
= she is a sorceress .
< she is a sorceress . <EOS>

> noi siamo indifesi .
= we re defenseless .
< we re helpless . <EOS>

> e impegnata con il suo lavoro .
= she s busy with her work .
< she s busy with her work . <EOS>

> stiamo ancora parlando .
= we re still talking .
< we re still talking . <EOS>

> si e fermata per fumare una sigaretta .
= she stopped to smoke a cigarette .
< she stopped to smoke a cigarette . <EOS>

> noi siamo pronte a tutto .
= we re ready for anything .
< we re ready for anythin

In [30]:
score

"BLEU Score: {'bleu': 0.5500218427694827, 'precisions': [0.6474795640326976, 0.5908043032786885, 0.5283490011750881, 0.4528236914600551], 'brevity_penalty': 1.0, 'length_ratio': 1.5149638802889578, 'translation_length': 8808, 'reference_length': 5814}"

In [ ]:
bleu - 0.55
Bleu -  это основанная на точности мера, которая варьируется от 0 до 1. Чем ближе значение к 1, тем лучше прогноз. Полученный результат хороший